# 第5章 一等函数
## 1 函数内省
列出常规对象没有而函数有的属性：

In [1]:
class C:
    pass

obj = C()

def func():
    pass

sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

## 2 从定位参数到仅限关键字参数

### 2.1 e.g. `tag`函数用于生成HTML标签

In [2]:
def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

传入单个定位参数，生成一个指定名称的空标签：

In [3]:
tag('br')

'<br />'

第一个参数后面的任意个参数会被`*content`捕获，存入一个元组：

In [4]:
tag('p', 'hello')

'<p>hello</p>'

`tag`函数签名中没有明确指定名称的关键字参数会被`**attrs`捕获，存入一个字典：

In [5]:
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

`cls`参数只能作为关键字参数传入：

In [6]:
print(tag('p', 'hello', 'world', cls='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


调用`tag`函数时，即便第一个定位参数也能作为关键字参数传入：

In [7]:
tag(content='testing', name="img")

'<img content="testing" />'

在`my_tag`前面加上`**`，字典中的所有元素作为单个参数传入，同名键会绑定到对应的具名参数上，余下的则被`**attrs`捕获：

In [8]:
my_tag = {'name': 'img',
          'title': 'Sunset Boulevard',
          'src': 'sunset.jpg',
          'cls': 'framed'}

tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

## 3 获取关于参数的信息
### 3.1 e.g. 在指定长度附近截断字符串的函数

In [9]:
def clip(text, max_len=80):
    """在max_len前面或后面的第一个空格处截断文本"""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:  # 没找到空格
        end = len(text)
    return text[:end].rstrip()

### 3.2 e.g. 提取关于函数参数的信息

In [10]:
clip.__defaults__

(80,)

In [11]:
clip.__code__

<code object clip at 0x7f826c6775d0, file "<ipython-input-9-476a9d424fed>", line 1>

In [12]:
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

In [13]:
clip.__code__.co_argcount

2

上述结果可以看出，这种组织信息的方式并不是最便利的。参数名称在`__code__.co_varnames`中， 不过里面还有函数定义体中创建的局部变量。因此，参数名称是前$N$个字符串，$N$的值由`__code__.co_argcount`确定。

顺便说一下，这里不包含前缀为`*`或`**`的变长参数。参数的默认值只能通过它们在`__defaults__`元组中的位置确定，因此要从后向前扫描才能把参数和默认值对应起来。在这个示例中`clip`函数有两个参数，`text`和`max_len`，其中一个有默认值，即80，因此它必然属于最后一个参数，即`max_len`。

幸好，我们有更好的方式——使用`inspect`模块，见下面的示例。

### 3.3 e.g. 提取函数的签名

In [14]:
from inspect import signature

sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [15]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

str(sig)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


'(text, max_len=80)'

`kind`属性的值是`_ParameterKind`类中的5个值之一：
- POSITIONAL_OR_KEYWORD
- VAR_POSITIONAL
- VAR_KEYWORD
- KEYWORD_ONLY
- POSITIONAL_ONLY

## 4 支持函数式编程的包
### 4.1 `operator`模块
在函数式编程中，经常需要把算术运算符当作函数使用。例如，不使用递归计算阶乘。operator模块提供了多个算数运算符对应的函数，另外还有能代替`lambda`读取元素或对象属性的`itemgetter`和`attrgetter`（会自行创建函数，类似`lambda`）等。

**V.S. lambda**  
[stackoverflow](https://stackoverflow.com/questions/2705104/lambda-vs-operator-attrgetterxxx-as-a-sort-key-function)说`itemgetter`等是调用对象的内置函数，速度上更快，而`lamdba`通过外部函数调用。

下面是`operator`模块中定义的部分函数：

In [16]:
import operator

print([name for name in dir(operator) if not name.startswith('_')])

['abs', 'add', 'and_', 'attrgetter', 'concat', 'contains', 'countOf', 'delitem', 'eq', 'floordiv', 'ge', 'getitem', 'gt', 'iadd', 'iand', 'iconcat', 'ifloordiv', 'ilshift', 'imatmul', 'imod', 'imul', 'index', 'indexOf', 'inv', 'invert', 'ior', 'ipow', 'irshift', 'is_', 'is_not', 'isub', 'itemgetter', 'itruediv', 'ixor', 'le', 'length_hint', 'lshift', 'lt', 'matmul', 'methodcaller', 'mod', 'mul', 'ne', 'neg', 'not_', 'or_', 'pos', 'pow', 'rshift', 'setitem', 'sub', 'truediv', 'truth', 'xor']


以`i`开头、后面是另一个运算符的那些名称（如`iadd`、`iand`等），对应的是增量赋值运算符（如`+=`、`&=`等）。

#### 4.1.1 e.g. 使用`reduce`函数和一个`lambda`计算阶乘

In [17]:
from functools import reduce

def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

fact(5)

120

#### 4.1.2 e.g. 使用`reduce`和`operator.mul`函数计算阶乘

In [18]:
from functools import reduce
from operator import mul


def fact(n):
    return reduce(mul, range(1, n+1))


fact(5)

120

#### 4.1.3 e.g. 使用`itemgetter`排序一个元组列表

In [19]:
from operator import itemgetter
metro_data = [('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
              ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
              ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
              ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
              ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)), ]

for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


如果把多个参数传给`itemgetter`，它构建的函数会返回提取的值构成的元组：

In [20]:
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


#### 4.1.4 e.g. 使用`attrgetter`处理`namedtuple`
`metro_data`同上

In [21]:
from collections import namedtuple

LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')

metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
               for name, cc, pop, (lat, long) in metro_data]
metro_areas

[Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667)),
 Metropolis(name='Delhi NCR', cc='IN', pop=21.935, coord=LatLong(lat=28.613889, long=77.208889)),
 Metropolis(name='Mexico City', cc='MX', pop=20.142, coord=LatLong(lat=19.433333, long=-99.133333)),
 Metropolis(name='New York-Newark', cc='US', pop=20.104, coord=LatLong(lat=40.808611, long=-74.020386)),
 Metropolis(name='Sao Paulo', cc='BR', pop=19.649, coord=LatLong(lat=-23.547778, long=-46.635833))]

按照纬度排序城市列表：

In [22]:
metro_areas[0].coord.lat

35.689722

In [23]:
from operator import attrgetter

# 定义一个attrgetter，获取name和嵌套的coord.lat
name_lat = attrgetter('name', 'coord.lat')

for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


其实`lambda`也能达到同样的功能：

In [24]:
for city in sorted(metro_areas, key=lambda x: x.coord.lat):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


#### 4.1.5 e.g. 使用methodcaller冻结参数
`methodcaller`允许使用字符串方式调用对象并冻结参数：

In [25]:
from operator import methodcaller

upcase = methodcaller('upper')
upcase('hello')

'HELLO'

In [26]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate('hello world')

'hello-world'

### 4.2 使用`functools.partial`冻结参数
TBD